# Digit Detection: LeNet

Students: Juan Sebastián Barreto Jimenez y Juan Camilo Devia Bastos

Consultant: Ing. Eduardo Andrés Gerlien Reyes

Client: Ing. Olga Lucía Quintero Montoya

## Avoid warnings

In [37]:
# Avoid warnings in terminal
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Libraries

In [38]:
from tensorflow import keras
import keras.layers as layers
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
import tensorflow as tf
import visualkeras

from keras.datasets import mnist
from tensorflow.keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
import numpy as np
import cv2

## Download and Process Data from MNIST Keras

In [39]:
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()
# if we are using "channels first" ordering, then reshape the
# design matrix such that the matrix is:
# num_samples x depth x rows x columns
if K.image_data_format() == "channels_first":
	trainData = trainData.reshape((trainData.shape[0], 1, 28, 28))
	testData = testData.reshape((testData.shape[0], 1, 28, 28))
# otherwise, we are using "channels last" ordering, so the design
# matrix shape should be: num_samples x rows x columns x depth
else:
	trainData = trainData.reshape((trainData.shape[0], 28, 28, 1))
	testData = testData.reshape((testData.shape[0], 28, 28, 1))
# scale data to the range of [0, 1]
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0
# transform the training and testing labels into vectors in the
# range [0, classes] -- this generates a vector for each label,
# where the index of the label is set to `1` and all other entries
# to `0`; in the case of MNIST, there are 10 class labels
trainLabels = np_utils.to_categorical(trainLabels, 10)
testLabels = np_utils.to_categorical(testLabels, 10)

## Export data to Matlab

In [40]:
((trainData_m, trainLabels_m), (testData_m, testLabel_m)) = mnist.load_data()
trainData_m = trainData_m.reshape((28, 28, 1, trainData_m.shape[0]))
testData_m = testData_m.reshape((28, 28, 1, testData.shape[0]))
# scale data to the range of [0, 1]
trainData_m = trainData_m.astype("float32") / 255.0
testData_m = testData_m.astype("float32") / 255.0

from scipy.io import savemat
m_dic = {"trainData": trainData_m, "testData": testData_m, "trainLabels": trainLabels_m, "testLabels": testLabel_m}
savemat("../LeNet_Matlab/matlab_matrix.mat", m_dic)

## Mount CNN: LeNet

In [41]:
model = keras.Sequential()
numChannels, imgRows, imgCols = 1, 28, 28
numClasses = 10
inputShape = (imgRows, imgCols, numChannels)
activation = 'relu' 

# if we are using "channels first", update the input shape
if K.image_data_format() == "channels_first":
	print("ENtre")
	inputShape = (numChannels, imgRows, imgCols)

model.add(layers.Conv2D(filters=20, kernel_size=5, padding="same", activation=activation, input_shape=inputShape))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(layers.Conv2D(filters=50, kernel_size=5, padding="same", activation=activation))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(layers.Dense(units=500, activation='relu'))
model.add(layers.Dense(units=numClasses, activation = 'softmax'))

## Compile CNN: LeNet

In [42]:
opt = SGD(learning_rate=0.01)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])

## Train model

In [43]:
print("[INFO] training...")
model.fit(trainData, trainLabels, batch_size=128, epochs=20,verbose=1)

[INFO] training...
Epoch 1/20
469/469 [==============================] - 49s 102ms/step - loss: 0.9979 - accuracy: 0.7167
Epoch 2/20
469/469 [==============================] - 49s 104ms/step - loss: 0.2643 - accuracy: 0.9215
Epoch 3/20
469/469 [==============================] - 49s 103ms/step - loss: 0.1865 - accuracy: 0.9444
Epoch 4/20
469/469 [==============================] - 43s 93ms/step - loss: 0.1460 - accuracy: 0.9571
Epoch 5/20
469/469 [==============================] - 44s 93ms/step - loss: 0.1198 - accuracy: 0.9649
Epoch 6/20
469/469 [==============================] - 48s 102ms/step - loss: 0.1022 - accuracy: 0.9696
Epoch 7/20
469/469 [==============================] - 46s 99ms/step - loss: 0.0909 - accuracy: 0.9724
Epoch 8/20
469/469 [==============================] - 46s 99ms/step - loss: 0.0811 - accuracy: 0.9758
Epoch 9/20
469/469 [==============================] - 45s 97ms/step - loss: 0.0735 - accuracy: 0.9782
Epoch 10/20
469/469 [==============================] - 46s 

## Evaluate model

In [46]:
print("[INFO] evaluating...")
(loss, accuracy) = model.evaluate(testData, testLabels,batch_size=128, verbose=1)
print("[INFO] accuracy: {:.2f}%".format(accuracy * 100))

[INFO] evaluating...
79/79 [==============================] - 2s 22ms/step - loss: 0.0374 - accuracy: 0.9873
[INFO] accuracy: 98.73%


## Save Weights

In [45]:
print("[INFO] dumping weights to file...")
model.save_weights('output/lenet_weights.hdf5', overwrite=True)

[INFO] dumping weights to file...


## Randomly select a few testing digits

In [46]:
for i in np.random.choice(np.arange(0, len(testLabels)), size=(10,)):
	# classify the digit
	probs = model.predict(testData[np.newaxis, i])
	prediction = probs.argmax(axis=1)
	# extract the image from the testData if using "channels_first"
	# ordering
	if K.image_data_format() == "channels_first":
		image = (testData[i][0] * 255).astype("uint8")
	# otherwise we are using "channels_last" ordering
	else:
		image = (testData[i] * 255).astype("uint8")
	# merge the channels into one image
	image = cv2.merge([image] * 3)
	# resize the image from a
	#  28 x 28 image to a 96 x 96 image so we
	# can better see it
	image = cv2.resize(image, (96, 96), interpolation=cv2.INTER_LINEAR)
	# show the image and prediction
	cv2.putText(image, str(prediction[0]), (5, 20),
				cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
	print("[INFO] Predicted: {}, Actual: {}".format(prediction[0],
		np.argmax(testLabels[i])))
	cv2.imshow("Digit", image)
	cv2.waitKey(0)

[INFO] Predicted: 4, Actual: 4


## Visualize CNN: LeNet

In [5]:
visualkeras.layered_view(model)

NameError: name 'model' is not defined